In [12]:
import os, sys, random
import numpy as np
import pandas as pd


from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings('default')


# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline


# 評価算出
from sklearn.metrics import f1_score


from google.colab import drive
from google.colab import files
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/SIGNITE/Predict_genre_music')
os.getcwd()

datapath = "data/Matsuda/high_score_feature/"
savepath="models/NN/"
submitcsv = 'data/datas/submit_2.csv'

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
categories = df_train.columns
categories

In [ ]:
df_train.dtypes

In [ ]:
df_train.drop('Unnamed: 0', axis=1)

In [ ]:
df_test.drop('Unnamed: 0', axis=1)

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data

from sklearn.preprocessing import LabelEncoder
import pandas as pd
from tqdm import tqdm
import pyarrow as pa

from models import Net

def mean_norm(df):
    """ 標準化 """
    return df.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

def preprocess(df_train):
    """ 前処理  """

    df = df_train.copy()

    # # ラベルエンコーディング
    # le1 = LabelEncoder()
    # encoded1 = le1.fit_transform(df['region'].values)
    # df['region'] = encoded1

    # le2 = LabelEncoder()
    # encoded2 = le2.fit_transform(df['tempo'].values)
    # df['tempo'] = encoded2


    # 欠損値を平均で補完
    # df = df.fillna(df.mean())

    # すべて0であればOK
    # print(df.isnull().sum())

    # データと正解ラベルを分割
    X, y = df.drop("genre", axis=1), df["genre"]

    # 標準化
    X = mean_norm(X)

    # numpyに変換
    X, y = X.values.astype(np.float32), y.values

    # 正解ラベルをone-hotに
    # n_labels = len(np.unique(y))
    # y = np.eye(n_labels)[y]

    return X, y

def make_dataloader(X, y, batch_size):
    """ Dataloader作成 """

    dataset = data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

    return dataloader

def train(model, dataloader, optimizer, criterion, num_epochs):
    """ 学習 """

    # ネットワークをGPUへ
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train() # 訓練モードに

    # lossやaccのプロット用
    history = {"loss": [], "acc": []}

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        epoch_loss = epoch_corrects = total = 0  

        # データローダーからミニバッチを取り出すループ
        for inputs, labels in tqdm(dataloader):

            inputs, labels = inputs.to(device), labels.to(device)

            # optimizerを初期化
            optimizer.zero_grad()

            # 順伝搬（forward）計算
            outputs, features = model(inputs)

            loss = criterion(outputs, labels)  # 損失を計算
            _, preds = torch.max(outputs, 1)  # ラベルを予測

            # print(preds)
            
            # 逆伝播（backward）計算
            loss.backward()
            optimizer.step()

            # イタレーション結果の計算
            mini_batch_size = inputs.size(0)

            epoch_loss += loss.item() * mini_batch_size
            epoch_corrects += torch.sum(preds == labels.data)
            total += mini_batch_size*1.0

        # epochごとのlossと正解率を表示
        epoch_loss = epoch_loss / total
        epoch_acc = epoch_corrects / total

        history["loss"].append(epoch_loss)
        history["acc"].append(epoch_acc)

        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))


if __name__ == '__main__':

    #　読み込み
    df_train = pd.read_csv(datapath + "train_m.csv")
    df_test = pd.read_csv(datapath + "test.csv")

    # 前処理
    X, y = preprocess(df_train)
    dataloader = make_dataloader(X, y, 100)

    # モデル呼び出し
    data_dim = X.shape[1]
    num_classes = 11
    model = Net(data_dim, num_classes)

    # 最適アルゴリズム
    lr = 0.00001
    beta1, beta2 = 0.0, 0.9
    optimizer = torch.optim.Adam(model.parameters(), lr, [beta1, beta2])

    # 損失関数
    criterion = nn.CrossEntropyLoss(reduction='mean')

    # 学習回数
    num_epochs = 100

    # 学習
    train(model, dataloader, optimizer, criterion, num_epochs)

    # モデル呼び出し
    # model_path = 'Model/NN/1.pth'
    # model.load_state_dict(torch.load(model_path))

    # モデル保存
    model_path = savepath + '1.pth'
    torch.save(model.state_dict(), model_path)

ModuleNotFoundError: No module named 'torch'

In [ ]:
datapath

In [ ]:
X

In [ ]:
df_train.columns

In [ ]:
df_train["genre"]